In [57]:
import pandas as pd

df = pd.read_csv("../src/taxipred/data/taxi_trip_pricing.csv")

df.head()


,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


## GENERAL

### Påverkar dessa priset:
- Time_of_Day
- Passenger_Count
- Traffic_Conditions
- Weather

De som inte påverkar priset kan vi droppa. 
De vi vill ha kvar skapar vi funktioner för att ersätta nullvärden (med hjälp av andra kolumner i data framen)
Om det finns två nullvärden inom de viktiga kolumnerna på en rad går det inte att räkna ut, eventuellt använda linear_regression?


- För att räkna ut Trip_Price: Per_Minute_Rate * Trip_Duration_Minutes + Trip_Duration_Minutes * Trip_Distance_km + Base_Fare
- För att räkna ut Base_Fare (om resten av värdena finns): Lägg samman Per_Minute_Rate * Trip_Duration_Minutes + Trip_Duration_Minutes * Trip_Distance_km, substrahera med Trip_Price
- 

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


In [59]:
rows_with_nulls = df[df.isnull().any(axis=1)]
rows_with_nulls

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180
6,3.85,Afternoon,Weekday,4.0,High,Rain,3.51,1.66,NaN,5.05,11.2645
7,43.44,Evening,Weekend,3.0,NaN,Clear,2.97,1.87,0.23,NaN,101.1216
...,...,...,...,...,...,...,...,...,...,...,...
991,35.04,Morning,Weekend,4.0,Medium,Rain,2.90,1.10,0.15,9.99,NaN
993,NaN,Morning,Weekday,3.0,Medium,Clear,2.65,1.35,NaN,25.61,55.3348
996,45.95,Night,Weekday,4.0,Medium,Clear,3.12,0.61,NaN,61.96,62.1295
997,7.70,Morning,Weekday,3.0,Low,Rain,2.08,1.78,NaN,54.18,33.1236


In [60]:
df

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180
...,...,...,...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,Weekend,4.0,Medium,Clear,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,Weekday,4.0,Medium,Clear,3.12,0.61,NaN,61.96,62.1295
997,7.70,Morning,Weekday,3.0,Low,Rain,2.08,1.78,NaN,54.18,33.1236
998,47.56,Morning,Weekday,1.0,Low,Clear,2.67,0.82,0.17,114.94,61.2090


In [61]:
df['null_count'] = df.isnull().sum(axis=1)
rows_with_multiple_nulls = df[df['null_count'] > 2]
rows_with_multiple_nulls

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,null_count
19,15.27,Morning,NaN,NaN,Low,Clear,3.93,0.73,0.12,NaN,27.3543,3
107,38.02,Evening,NaN,4.0,NaN,Clear,NaN,1.31,0.35,33.73,66.2817,3
125,21.93,Afternoon,NaN,3.0,Low,NaN,4.53,0.76,NaN,74.52,48.0240,3
137,NaN,NaN,NaN,3.0,Low,Clear,4.52,1.38,0.35,57.56,73.5870,3
173,12.10,Morning,Weekend,NaN,Low,Clear,NaN,0.59,0.31,NaN,19.2643,3
177,19.95,Morning,Weekday,3.0,Low,NaN,2.50,0.88,0.49,NaN,NaN,3
262,7.86,Afternoon,NaN,3.0,Medium,Rain,4.82,0.75,0.26,NaN,NaN,3
344,NaN,Evening,NaN,NaN,Low,Clear,4.62,1.86,0.43,7.19,45.8975,3
583,17.11,Afternoon,Weekday,NaN,Medium,NaN,2.77,1.23,NaN,91.82,36.6701,3
631,44.87,Afternoon,NaN,4.0,Low,Clear,4.27,NaN,0.34,90.81,NaN,3


## DROP UNNECESSARY COLUMNS

- VILKA AV DESSA?!

In [62]:
# Spara:   "Time_of_Day": för att addera en procentuell ökning beroende på rusningstrafik(appen känner av datetime.now()), 
#          "Day_of_Week": Kolla om det blir dyrare på helgdagar eller veckodagar


# Kan behöva spara dessa för att de kan påverka hur lång tid resan tar, alltså även slutpriset
# Använd correlation heatmap på dessa
# Få ihop hela flödet och förfina sedan
# Exportera den rensade csv-filen i stället för att göra all transformation i en .py-fil
# Lägg till egna grejer om man vill, bara bra!

df = df.drop(["Passenger_Count", "Traffic_Conditions", "Weather", "Day_of_Week"], axis=1)
df

,Trip_Distance_km,Time_of_Day,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,null_count
0,19.35,Morning,3.56,0.80,0.32,53.82,36.2624,0
1,47.59,Afternoon,NaN,0.62,0.43,40.57,NaN,2
2,36.87,Evening,2.70,1.21,0.15,37.27,52.9032,0
3,30.33,Evening,3.48,0.51,0.15,116.81,36.4698,1
4,NaN,Evening,2.93,0.63,0.32,22.64,15.6180,1
...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,2.39,0.62,0.49,58.39,34.4049,0
996,45.95,Night,3.12,0.61,NaN,61.96,62.1295,1
997,7.70,Morning,2.08,1.78,NaN,54.18,33.1236,1
998,47.56,Morning,2.67,0.82,0.17,114.94,61.2090,0


### Ta bort de rader som innehåller fler än ett nullvärde

In [63]:
df = df[df['null_count'] < 2]
df

,Trip_Distance_km,Time_of_Day,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price,null_count
0,19.35,Morning,3.56,0.80,0.32,53.82,36.2624,0
2,36.87,Evening,2.70,1.21,0.15,37.27,52.9032,0
3,30.33,Evening,3.48,0.51,0.15,116.81,36.4698,1
4,NaN,Evening,2.93,0.63,0.32,22.64,15.6180,1
5,8.64,Afternoon,2.55,1.71,0.48,89.33,60.2028,0
...,...,...,...,...,...,...,...,...
995,5.49,Afternoon,2.39,0.62,0.49,58.39,34.4049,0
996,45.95,Night,3.12,0.61,NaN,61.96,62.1295,1
997,7.70,Morning,2.08,1.78,NaN,54.18,33.1236,1
998,47.56,Morning,2.67,0.82,0.17,114.94,61.2090,0


In [64]:
df = df.drop('null_count', axis=1)
df

,Trip_Distance_km,Time_of_Day,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,3.56,0.80,0.32,53.82,36.2624
2,36.87,Evening,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,2.93,0.63,0.32,22.64,15.6180
5,8.64,Afternoon,2.55,1.71,0.48,89.33,60.2028
...,...,...,...,...,...,...,...
995,5.49,Afternoon,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,3.12,0.61,NaN,61.96,62.1295
997,7.70,Morning,2.08,1.78,NaN,54.18,33.1236
998,47.56,Morning,2.67,0.82,0.17,114.94,61.2090


### När vi nu har de viktiga kolumnerna, ersätt NaN-värden med värden som räknas ihop baserat på övrig info

## Per_Minute_Rate:

Per_Minute_Rate = (Trip_Price - Base_Fare - (Trip_Distance_km * Per_Km_Rate)) / Trip_Duration_Minutes

In [65]:
# STEG 1: AUTOMATISK IDENTIFIERING
# Här skapar vi en "mask" som automatiskt hittar alla rader där 'Per_Minute_Rate' är NaN.
# df['Per_Minute_Rate'].isnull() returnerar True för varje rad som har ett NaN-värde och False annars.
# Vi lägger också till ett villkor för att undvika division med noll.
mask_for_nan = (df['Per_Minute_Rate'].isnull()) & (df['Trip_Duration_Minutes'] > 0)

# STEG 2: BERÄKNING PÅ ALLA IDENTIFIERADE RADER
# Nu använder vi masken för att beräkna det nya värdet.
# Pandas är smart nog att bara utföra beräkningen på de rader där masken är 'True'.
# Detta sker för alla identifierade rader samtidigt, inte en i taget.
numerator = (df.loc[mask_for_nan, 'Trip_Price'] -
             df.loc[mask_for_nan, 'Base_Fare'] -
             (df.loc[mask_for_nan, 'Trip_Distance_km'] * df.loc[mask_for_nan, 'Per_Km_Rate']))

calculated_rate = numerator / df.loc[mask_for_nan, 'Trip_Duration_Minutes']

# STEG 3: UPPDATERING AV ALLA IDENTIFIERADE RADER
# Till sist använder vi samma mask för att tilldela de nya, beräknade värdena.
# Endast cellerna i 'Per_Minute_Rate'-kolumnen på de rader där masken är 'True' kommer att uppdateras.
df.loc[mask_for_nan, 'Per_Minute_Rate'] = calculated_rate

# print(df)


In [66]:
df

,Trip_Distance_km,Time_of_Day,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,3.56,0.80,0.32,53.82,36.2624
2,36.87,Evening,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,2.93,0.63,0.32,22.64,15.6180
5,8.64,Afternoon,2.55,1.71,0.48,89.33,60.2028
...,...,...,...,...,...,...,...
995,5.49,Afternoon,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,3.12,0.61,0.50,61.96,62.1295
997,7.70,Morning,2.08,1.78,0.32,54.18,33.1236
998,47.56,Morning,2.67,0.82,0.17,114.94,61.2090


## Per_Km_Rate

In [67]:
# skapa en variabel som tar ut de som saknar tri_duration_minute och som har en per_minute_rate större än 0

mask_for_nan = (df['Trip_Duration_Minutes'].isnull()) & (df['Per_Minute_Rate'] > 0)

numerator = (df.loc[mask_for_nan, 'Trip_Price'] -
             df.loc[mask_for_nan, 'Base_Fare'] -
             (df.loc[mask_for_nan, 'Trip_Distance_km'] * df.loc[mask_for_nan, 'Per_Km_Rate']))

calculated_duration = numerator / df.loc[mask_for_nan, 'Per_Minute_Rate']

df.loc[mask_for_nan, 'Trip_Duration_Minutes'] = calculated_duration
print(df)

     Trip_Distance_km Time_of_Day  Base_Fare  Per_Km_Rate  Per_Minute_Rate  \
0               19.35     Morning       3.56         0.80             0.32   
2               36.87     Evening       2.70         1.21             0.15   
3               30.33     Evening       3.48         0.51             0.15   
4                 NaN     Evening       2.93         0.63             0.32   
5                8.64   Afternoon       2.55         1.71             0.48   
..                ...         ...        ...          ...              ...   
995              5.49   Afternoon       2.39         0.62             0.49   
996             45.95       Night       3.12         0.61             0.50   
997              7.70     Morning       2.08         1.78             0.32   
998             47.56     Morning       2.67         0.82             0.17   
999             22.85     Morning       4.34          NaN             0.23   

     Trip_Duration_Minutes  Trip_Price  
0                    5

In [68]:
nan_count = df['Trip_Duration_Minutes'].isnull().sum()
print(nan_count)

0


In [69]:
df.isnull().sum()

Trip_Distance_km         33
Time_of_Day              39
Base_Fare                31
Per_Km_Rate              30
Per_Minute_Rate           0
Trip_Duration_Minutes     0
Trip_Price               26
dtype: int64

In [70]:
df

,Trip_Distance_km,Time_of_Day,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,3.56,0.80,0.32,53.82,36.2624
2,36.87,Evening,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,2.93,0.63,0.32,22.64,15.6180
5,8.64,Afternoon,2.55,1.71,0.48,89.33,60.2028
...,...,...,...,...,...,...,...
995,5.49,Afternoon,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,3.12,0.61,0.50,61.96,62.1295
997,7.70,Morning,2.08,1.78,0.32,54.18,33.1236
998,47.56,Morning,2.67,0.82,0.17,114.94,61.2090


## Trip_Distance_km

In [71]:
# Trip_Price = BaseFare + (Trip_Duration_Minutes * Per_Minute_Rate) + (Trip_Distance_km * Per_Km_Rate)
# Trip_Distance_km = BaseFare + (Trip_Duration_Minutes * Per_Minute_Rate). Ta det som återstår av värdet och dividera med per_km_rate

mask_for_nan = (df['Trip_Distance_km'].isnull()) & (df['Per_Km_Rate'] > 0)

numerator = (df.loc[mask_for_nan, 'Trip_Price'] -
             df.loc[mask_for_nan, 'Base_Fare'] -
             (df.loc[mask_for_nan, 'Trip_Duration_Minutes'] * df.loc[mask_for_nan, 'Per_Minute_Rate']))

# Alltså dela det vi har kvar med Per_Km_Rate
calculated_distance = numerator / df.loc[mask_for_nan, 'Per_Km_Rate']

df.loc[mask_for_nan, 'Trip_Distance_km'] = calculated_distance
print(df)

     Trip_Distance_km Time_of_Day  Base_Fare  Per_Km_Rate  Per_Minute_Rate  \
0               19.35     Morning       3.56         0.80             0.32   
2               36.87     Evening       2.70         1.21             0.15   
3               30.33     Evening       3.48         0.51             0.15   
4                8.64     Evening       2.93         0.63             0.32   
5                8.64   Afternoon       2.55         1.71             0.48   
..                ...         ...        ...          ...              ...   
995              5.49   Afternoon       2.39         0.62             0.49   
996             45.95       Night       3.12         0.61             0.50   
997              7.70     Morning       2.08         1.78             0.32   
998             47.56     Morning       2.67         0.82             0.17   
999             22.85     Morning       4.34          NaN             0.23   

     Trip_Duration_Minutes  Trip_Price  
0                    5

In [72]:
df.isnull().sum()

Trip_Distance_km          0
Time_of_Day              39
Base_Fare                31
Per_Km_Rate              30
Per_Minute_Rate           0
Trip_Duration_Minutes     0
Trip_Price               26
dtype: int64

In [73]:
df

,Trip_Distance_km,Time_of_Day,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,3.56,0.80,0.32,53.82,36.2624
2,36.87,Evening,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,3.48,0.51,0.15,116.81,36.4698
4,8.64,Evening,2.93,0.63,0.32,22.64,15.6180
5,8.64,Afternoon,2.55,1.71,0.48,89.33,60.2028
...,...,...,...,...,...,...,...
995,5.49,Afternoon,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,3.12,0.61,0.50,61.96,62.1295
997,7.70,Morning,2.08,1.78,0.32,54.18,33.1236
998,47.56,Morning,2.67,0.82,0.17,114.94,61.2090


## Per_Km_Rate

In [74]:
# Trip_Price = BaseFare + (Trip_Duration_Minutes * Per_Minute_Rate) + (Trip_Distance_km * Per_Km_Rate)
# Per_km_Rate = [Trip_Price - BaseFare - (Trip_Duration_Minutes * Per_Minute_Rate)] / Trip_Distance_km

mask_for_nan = (df['Per_Km_Rate'].isnull()) & (df['Trip_Distance_km'] > 0)

# Täljare
numerator = (df.loc[mask_for_nan, 'Trip_Price'] -
             df.loc[mask_for_nan, 'Base_Fare'] -
             (df.loc[mask_for_nan, 'Trip_Duration_Minutes'] * df.loc[mask_for_nan, 'Per_Minute_Rate']))

calculated_rate = numerator / df.loc[mask_for_nan, 'Trip_Distance_km']

# Fyll i de saknade värdena i Per_km_Rate-kolumnen
df.loc[mask_for_nan, 'Per_Km_Rate'] = calculated_rate

print(df)

     Trip_Distance_km Time_of_Day  Base_Fare  Per_Km_Rate  Per_Minute_Rate  \
0               19.35     Morning       3.56         0.80             0.32   
2               36.87     Evening       2.70         1.21             0.15   
3               30.33     Evening       3.48         0.51             0.15   
4                8.64     Evening       2.93         0.63             0.32   
5                8.64   Afternoon       2.55         1.71             0.48   
..                ...         ...        ...          ...              ...   
995              5.49   Afternoon       2.39         0.62             0.49   
996             45.95       Night       3.12         0.61             0.50   
997              7.70     Morning       2.08         1.78             0.32   
998             47.56     Morning       2.67         0.82             0.17   
999             22.85     Morning       4.34         1.50             0.23   

     Trip_Duration_Minutes  Trip_Price  
0                    5

In [75]:
df.isnull().sum()

Trip_Distance_km          0
Time_of_Day              39
Base_Fare                31
Per_Km_Rate               0
Per_Minute_Rate           0
Trip_Duration_Minutes     0
Trip_Price               26
dtype: int64

## Base_Fare

In [76]:
# Trip_Price = BaseFare + (Trip_Duration_Minutes * Per_Minute_Rate) + (Trip_Distance_km * Per_Km_Rate)
# Base_Fare = [Trip_Price - (Trip_Duration_Minutes * Per_Minute_Rate)] - [Trip_Distance_km * Per_Km_Rate]

mask_for_nan = df['Base_Fare'].isnull()

calculated_base_fare = (df.loc[mask_for_nan, 'Trip_Price'] -
                        (df.loc[mask_for_nan, 'Trip_Duration_Minutes'] * df.loc[mask_for_nan, 'Per_Minute_Rate']) -
                        (df.loc[mask_for_nan, 'Trip_Distance_km'] * df.loc[mask_for_nan, 'Per_Km_Rate']))

df.loc[mask_for_nan, 'Base_Fare'] = calculated_base_fare

print(df)

     Trip_Distance_km Time_of_Day  Base_Fare  Per_Km_Rate  Per_Minute_Rate  \
0               19.35     Morning       3.56         0.80             0.32   
2               36.87     Evening       2.70         1.21             0.15   
3               30.33     Evening       3.48         0.51             0.15   
4                8.64     Evening       2.93         0.63             0.32   
5                8.64   Afternoon       2.55         1.71             0.48   
..                ...         ...        ...          ...              ...   
995              5.49   Afternoon       2.39         0.62             0.49   
996             45.95       Night       3.12         0.61             0.50   
997              7.70     Morning       2.08         1.78             0.32   
998             47.56     Morning       2.67         0.82             0.17   
999             22.85     Morning       4.34         1.50             0.23   

     Trip_Duration_Minutes  Trip_Price  
0                    5

In [77]:
df.isnull().sum()

Trip_Distance_km          0
Time_of_Day              39
Base_Fare                 0
Per_Km_Rate               0
Per_Minute_Rate           0
Trip_Duration_Minutes     0
Trip_Price               26
dtype: int64

## Trip_Price

In [78]:
# Trip_Price = BaseFare + (Trip_Duration_Minutes * Per_Minute_Rate) + (Trip_Distance_km * Per_Km_Rate)
# Base_Fare = [Trip_Price - (Trip_Duration_Minutes * Per_Minute_Rate)] - [Trip_Distance_km * Per_Km_Rate]

mask_for_nan = df['Trip_Price'].isnull()

calculated_trip_price = (df.loc[mask_for_nan, 'Base_Fare'] +
                        (df.loc[mask_for_nan, 'Trip_Duration_Minutes'] * df.loc[mask_for_nan, 'Per_Minute_Rate']) +
                        (df.loc[mask_for_nan, 'Trip_Distance_km'] * df.loc[mask_for_nan, 'Per_Km_Rate']))

df.loc[mask_for_nan, 'Trip_Price'] = calculated_trip_price

print(df)

     Trip_Distance_km Time_of_Day  Base_Fare  Per_Km_Rate  Per_Minute_Rate  \
0               19.35     Morning       3.56         0.80             0.32   
2               36.87     Evening       2.70         1.21             0.15   
3               30.33     Evening       3.48         0.51             0.15   
4                8.64     Evening       2.93         0.63             0.32   
5                8.64   Afternoon       2.55         1.71             0.48   
..                ...         ...        ...          ...              ...   
995              5.49   Afternoon       2.39         0.62             0.49   
996             45.95       Night       3.12         0.61             0.50   
997              7.70     Morning       2.08         1.78             0.32   
998             47.56     Morning       2.67         0.82             0.17   
999             22.85     Morning       4.34         1.50             0.23   

     Trip_Duration_Minutes  Trip_Price  
0                    5

In [79]:
df.isnull().sum()

Trip_Distance_km          0
Time_of_Day              39
Base_Fare                 0
Per_Km_Rate               0
Per_Minute_Rate           0
Trip_Duration_Minutes     0
Trip_Price                0
dtype: int64

## Nu har jag värden i alla numeriska kolumner

In [80]:
df = df.dropna(subset=['Time_of_Day'])
df

,Trip_Distance_km,Time_of_Day,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,3.56,0.80,0.32,53.82,36.2624
2,36.87,Evening,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,3.48,0.51,0.15,116.81,36.4698
4,8.64,Evening,2.93,0.63,0.32,22.64,15.6180
5,8.64,Afternoon,2.55,1.71,0.48,89.33,60.2028
...,...,...,...,...,...,...,...
995,5.49,Afternoon,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,3.12,0.61,0.50,61.96,62.1295
997,7.70,Morning,2.08,1.78,0.32,54.18,33.1236
998,47.56,Morning,2.67,0.82,0.17,114.94,61.2090


In [81]:
df.isnull().sum()

Trip_Distance_km         0
Time_of_Day              0
Base_Fare                0
Per_Km_Rate              0
Per_Minute_Rate          0
Trip_Duration_Minutes    0
Trip_Price               0
dtype: int64

# WOHO!!

### EXPORT CSV-FILE

In [82]:
# Exporterar din DataFrame till en ny CSV-fil
df.to_csv('cleaned_data.csv', index=False)

In [83]:
df

,Trip_Distance_km,Time_of_Day,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,3.56,0.80,0.32,53.82,36.2624
2,36.87,Evening,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,3.48,0.51,0.15,116.81,36.4698
4,8.64,Evening,2.93,0.63,0.32,22.64,15.6180
5,8.64,Afternoon,2.55,1.71,0.48,89.33,60.2028
...,...,...,...,...,...,...,...
995,5.49,Afternoon,2.39,0.62,0.49,58.39,34.4049
996,45.95,Night,3.12,0.61,0.50,61.96,62.1295
997,7.70,Morning,2.08,1.78,0.32,54.18,33.1236
998,47.56,Morning,2.67,0.82,0.17,114.94,61.2090


In [84]:
df["Time_of_Day"].unique()

array(['Morning', 'Evening', 'Afternoon', 'Night'], dtype=object)